In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import sys
import psutil  

# Notebook options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Data Paths
flights_cleaned_path = 'resources/flights_cleaned.csv'
flights_cleaned_no_null_path = 'resources/flights_cleaned_no_null.csv'

print(psutil.virtual_memory()) 

svmem(total=17137168384, available=13073522688, percent=23.7, used=4063645696, free=13073522688)


In [2]:
# Load data from resource file

df = pd.read_csv(flights_cleaned_no_null_path, low_memory=False)
df.head(5)

,Unnamed: 0,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,27,1,1,4,NK,597,N528NK,MSP,FLL,115,12.0,220.0,166.0,1487,527.0,40.0,542,607.0,25.0,25.0,0.0,0.0,0.0,0.0
1,30,1,1,4,NK,168,N629NK,PHX,ORD,125,72.0,175.0,156.0,1440,622.0,10.0,549,632.0,43.0,43.0,0.0,0.0,0.0,0.0
2,35,1,1,4,HA,17,N389HA,LAS,HNL,145,0.0,385.0,361.0,2762,602.0,8.0,555,610.0,15.0,0.0,0.0,15.0,0.0,0.0
3,50,1,1,4,B6,1030,N239JB,BQN,MCO,307,-3.0,196.0,160.0,1129,509.0,11.0,500,520.0,20.0,20.0,0.0,0.0,0.0,0.0
4,52,1,1,4,B6,2134,N307JB,SJU,MCO,400,95.0,175.0,163.0,1189,727.0,3.0,605,730.0,85.0,0.0,0.0,85.0,0.0,0.0


In [3]:
print(psutil.virtual_memory())

svmem(total=17137168384, available=12886421504, percent=24.8, used=4250746880, free=12886421504)


In [4]:
df.describe()

,Unnamed: 0,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_DELAY,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06
mean,2.765490e+06,6.234027e+00,1.556726e+01,3.877865e+00,2.192196e+03,1.467735e+03,5.494791e+01,1.469302e+02,1.174160e+02,8.326045e+02,1.579944e+03,8.788255e+00,1.620392e+03,1.576297e+03,5.891440e+01,1.348057e+01,7.615387e-02,1.896955e+01,2.347284e+01,2.915290e+00
std,1.694721e+06,3.411281e+00,8.794511e+00,1.979265e+00,1.784775e+03,4.553922e+02,6.721908e+01,7.738762e+01,7.416358e+01,6.067612e+02,5.986072e+02,8.943844e+00,5.058204e+02,6.108446e+02,6.425032e+01,2.800368e+01,2.143460e+00,4.816164e+01,4.319702e+01,2.043334e+01
min,2.700000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-3.200000e+01,1.600000e+01,8.000000e+00,3.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.277126e+06,3.000000e+00,8.000000e+00,2.000000e+00,7.230000e+02,1.120000e+03,1.700000e+01,9.000000e+01,6.200000e+01,3.830000e+02,1.242000e+03,4.000000e+00,1.300000e+03,1.239000e+03,2.300000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.706132e+06,6.000000e+00,1.600000e+01,4.000000e+00,1.667000e+03,1.521000e+03,3.700000e+01,1.290000e+02,9.900000e+01,6.740000e+02,1.718000e+03,6.000000e+00,1.715000e+03,1.719000e+03,3.700000e+01,2.000000e+00,0.000000e+00,2.000000e+00,3.000000e+00,0.000000e+00
75%,4.071042e+06,9.000000e+00,2.300000e+01,5.000000e+00,3.289000e+03,1.835000e+03,7.000000e+01,1.810000e+02,1.490000e+02,1.069000e+03,2.040000e+03,9.000000e+00,2.027000e+03,2.044000e+03,7.000000e+01,1.800000e+01,0.000000e+00,1.900000e+01,2.900000e+01,0.000000e+00
max,5.819071e+06,1.200000e+01,3.100000e+01,7.000000e+00,9.320000e+03,2.359000e+03,1.988000e+03,7.660000e+02,6.900000e+02,4.983000e+03,2.400000e+03,2.480000e+02,2.400000e+03,2.400000e+03,1.971000e+03,1.134000e+03,5.730000e+02,1.971000e+03,1.331000e+03,1.211000e+03


In [5]:
# DF with features for first ml exploration


df_ml = df[['DEPARTURE_DELAY', 'MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DISTANCE']].copy()
df_ml.head(5)

,DEPARTURE_DELAY,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DISTANCE
0,12.0,1,4,NK,MSP,FLL,115,1487
1,72.0,1,4,NK,PHX,ORD,125,1440
2,0.0,1,4,HA,LAS,HNL,145,2762
3,-3.0,1,4,B6,BQN,MCO,307,1129
4,95.0,1,4,B6,SJU,MCO,400,1189


In [6]:
sys.getsizeof(df)

427740862

In [7]:
print(psutil.virtual_memory())

svmem(total=17137168384, available=12808855552, percent=25.3, used=4328312832, free=12808855552)


In [8]:
# Release original df dataframe from memory

del df
gc.collect()
#%reset Out


87

In [9]:
print(psutil.virtual_memory()) 

svmem(total=17137168384, available=12809646080, percent=25.3, used=4327522304, free=12809646080)


In [10]:
categorical_data = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
df_ml[categorical_data].nunique()

AIRLINE                 14
ORIGIN_AIRPORT         625
DESTINATION_AIRPORT    625
dtype: int64

In [11]:
og_counts = df_ml.ORIGIN_AIRPORT.value_counts()
dest_counts = df_ml.DESTINATION_AIRPORT.value_counts()

print(og_counts)

ORD      66663
ATL      56462
DFW      50478
DEN      43331
LAX      40281
IAH      30690
SFO      29534
PHX      27427
LAS      27225
LGA      22709
MCO      22349
BOS      21363
EWR      21163
CLT      20623
JFK      20260
BWI      19049
DTW      18956
SEA      18554
MSP      18171
MDW      17273
MIA      15785
PHL      14173
FLL      13712
DAL      13176
SLC      13086
DCA      13081
SAN      12270
TPA      11395
HOU      11138
STL       8673
OAK       8582
BNA       8427
AUS       7153
IAD       7011
SJC       6769
SMF       6562
MSY       6473
MCI       6460
PDX       6382
CLE       5790
RDU       5716
SNA       5438
MKE       4991
RSW       4851
PBI       4818
HNL       4639
SAT       4458
SJU       4308
CMH       4282
IND       4080
CVG       3790
13930     3650
PIT       3439
ABQ       3417
ONT       2991
10397     2946
BDL       2909
JAX       2892
RIC       2784
OKC       2775
OMA       2761
11298     2760
OGG       2711
BUR       2693
BUF       2530
MEM       2499
12892     

In [12]:
# Bucket number of unique origin / destination airports with under 1000 values
# Memory issues when encoding dataset. box sizes increased to limit of memory 

# Get app types
to_replace = og_counts[og_counts < 5000].index
to_replace_2 = dest_counts[dest_counts < 5000].index

# Loop through apps that are to be replaced, replacing in charity_df
for air_port in to_replace:
    df_ml.ORIGIN_AIRPORT = df_ml.ORIGIN_AIRPORT.replace(air_port, 'Other')
    
for air_port in to_replace_2:
    df_ml.DESTINATION_AIRPORT = df_ml.DESTINATION_AIRPORT.replace(air_port, 'Other')

In [13]:
df_ml.ORIGIN_AIRPORT.value_counts()

Other    263626
ORD       66663
ATL       56462
DFW       50478
DEN       43331
LAX       40281
IAH       30690
SFO       29534
PHX       27427
LAS       27225
LGA       22709
MCO       22349
BOS       21363
EWR       21163
CLT       20623
JFK       20260
BWI       19049
DTW       18956
SEA       18554
MSP       18171
MDW       17273
MIA       15785
PHL       14173
FLL       13712
DAL       13176
SLC       13086
DCA       13081
SAN       12270
TPA       11395
HOU       11138
STL        8673
OAK        8582
BNA        8427
AUS        7153
IAD        7011
SJC        6769
SMF        6562
MSY        6473
MCI        6460
PDX        6382
CLE        5790
RDU        5716
SNA        5438
Name: ORIGIN_AIRPORT, dtype: int64

In [14]:
# One Hot encoder,  label encoding

enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_ml[categorical_data]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(categorical_data)

In [15]:
print(psutil.virtual_memory()) 

svmem(total=17137168384, available=11873914880, percent=30.7, used=5263253504, free=11873914880)


In [16]:
# Merge encoded df and original df, remove pre-encoded columns

df_ml.drop(columns=categorical_data, inplace=True)
df_ml = df_ml.merge(encode_df, left_index=True, right_index=True)
df_ml.head()

,DEPARTURE_DELAY,MONTH,DAY_OF_WEEK,SCHEDULED_DEPARTURE,DISTANCE,AIRLINE_AA,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_AUS,ORIGIN_AIRPORT_BNA,ORIGIN_AIRPORT_BOS,ORIGIN_AIRPORT_BWI,ORIGIN_AIRPORT_CLE,ORIGIN_AIRPORT_CLT,ORIGIN_AIRPORT_DAL,ORIGIN_AIRPORT_DCA,ORIGIN_AIRPORT_DEN,ORIGIN_AIRPORT_DFW,ORIGIN_AIRPORT_DTW,ORIGIN_AIRPORT_EWR,ORIGIN_AIRPORT_FLL,ORIGIN_AIRPORT_HOU,ORIGIN_AIRPORT_IAD,ORIGIN_AIRPORT_IAH,ORIGIN_AIRPORT_JFK,ORIGIN_AIRPORT_LAS,ORIGIN_AIRPORT_LAX,ORIGIN_AIRPORT_LGA,ORIGIN_AIRPORT_MCI,ORIGIN_AIRPORT_MCO,ORIGIN_AIRPORT_MDW,ORIGIN_AIRPORT_MIA,ORIGIN_AIRPORT_MSP,ORIGIN_AIRPORT_MSY,ORIGIN_AIRPORT_OAK,ORIGIN_AIRPORT_ORD,ORIGIN_AIRPORT_Other,ORIGIN_AIRPORT_PDX,ORIGIN_AIRPORT_PHL,ORIGIN_AIRPORT_PHX,ORIGIN_AIRPORT_RDU,ORIGIN_AIRPORT_SAN,ORIGIN_AIRPORT_SEA,ORIGIN_AIRPORT_SFO,ORIGIN_AIRPORT_SJC,ORIGIN_AIRPORT_SLC,ORIGIN_AIRPORT_SMF,ORIGIN_AIRPORT_SNA,ORIGIN_AIRPORT_STL,ORIGIN_AIRPORT_TPA,DESTINATION_AIRPORT_ATL,DESTINATION_AIRPORT_AUS,DESTINATION_AIRPORT_BNA,DESTINATION_AIRPORT_BOS,DESTINATION_AIRPORT_BWI,DESTINATION_AIRPORT_CLE,DESTINATION_AIRPORT_CLT,DESTINATION_AIRPORT_DAL,DESTINATION_AIRPORT_DCA,DESTINATION_AIRPORT_DEN,DESTINATION_AIRPORT_DFW,DESTINATION_AIRPORT_DTW,DESTINATION_AIRPORT_EWR,DESTINATION_AIRPORT_FLL,DESTINATION_AIRPORT_HNL,DESTINATION_AIRPORT_HOU,DESTINATION_AIRPORT_IAD,DESTINATION_AIRPORT_IAH,DESTINATION_AIRPORT_JFK,DESTINATION_AIRPORT_LAS,DESTINATION_AIRPORT_LAX,DESTINATION_AIRPORT_LGA,DESTINATION_AIRPORT_MCI,DESTINATION_AIRPORT_MCO,DESTINATION_AIRPORT_MDW,DESTINATION_AIRPORT_MIA,DESTINATION_AIRPORT_MKE,DESTINATION_AIRPORT_MSP,DESTINATION_AIRPORT_MSY,DESTINATION_AIRPORT_OAK,DESTINATION_AIRPORT_ORD,DESTINATION_AIRPORT_Other,DESTINATION_AIRPORT_PBI,DESTINATION_AIRPORT_PDX,DESTINATION_AIRPORT_PHL,DESTINATION_AIRPORT_PHX,DESTINATION_AIRPORT_RDU,DESTINATION_AIRPORT_RSW,DESTINATION_AIRPORT_SAN,DESTINATION_AIRPORT_SAT,DESTINATION_AIRPORT_SEA,DESTINATION_AIRPORT_SFO,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_TPA
0,12.0,1,4,115,1487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,72.0,1,4,125,1440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1,4,145,2762,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-3.0,1,4,307,1129,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,95.0,1,4

In [17]:
print(psutil.virtual_memory())

svmem(total=17137168384, available=10935689216, percent=36.2, used=6201479168, free=10935689216)


In [18]:
# Free up memory

del encode_df
print(psutil.virtual_memory()) 

svmem(total=17137168384, available=11838570496, percent=30.9, used=5298597888, free=11838570496)


In [19]:
df_ml.head()

,DEPARTURE_DELAY,MONTH,DAY_OF_WEEK,SCHEDULED_DEPARTURE,DISTANCE,AIRLINE_AA,AIRLINE_AS,AIRLINE_B6,AIRLINE_DL,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN,ORIGIN_AIRPORT_ATL,ORIGIN_AIRPORT_AUS,ORIGIN_AIRPORT_BNA,ORIGIN_AIRPORT_BOS,ORIGIN_AIRPORT_BWI,ORIGIN_AIRPORT_CLE,ORIGIN_AIRPORT_CLT,ORIGIN_AIRPORT_DAL,ORIGIN_AIRPORT_DCA,ORIGIN_AIRPORT_DEN,ORIGIN_AIRPORT_DFW,ORIGIN_AIRPORT_DTW,ORIGIN_AIRPORT_EWR,ORIGIN_AIRPORT_FLL,ORIGIN_AIRPORT_HOU,ORIGIN_AIRPORT_IAD,ORIGIN_AIRPORT_IAH,ORIGIN_AIRPORT_JFK,ORIGIN_AIRPORT_LAS,ORIGIN_AIRPORT_LAX,ORIGIN_AIRPORT_LGA,ORIGIN_AIRPORT_MCI,ORIGIN_AIRPORT_MCO,ORIGIN_AIRPORT_MDW,ORIGIN_AIRPORT_MIA,ORIGIN_AIRPORT_MSP,ORIGIN_AIRPORT_MSY,ORIGIN_AIRPORT_OAK,ORIGIN_AIRPORT_ORD,ORIGIN_AIRPORT_Other,ORIGIN_AIRPORT_PDX,ORIGIN_AIRPORT_PHL,ORIGIN_AIRPORT_PHX,ORIGIN_AIRPORT_RDU,ORIGIN_AIRPORT_SAN,ORIGIN_AIRPORT_SEA,ORIGIN_AIRPORT_SFO,ORIGIN_AIRPORT_SJC,ORIGIN_AIRPORT_SLC,ORIGIN_AIRPORT_SMF,ORIGIN_AIRPORT_SNA,ORIGIN_AIRPORT_STL,ORIGIN_AIRPORT_TPA,DESTINATION_AIRPORT_ATL,DESTINATION_AIRPORT_AUS,DESTINATION_AIRPORT_BNA,DESTINATION_AIRPORT_BOS,DESTINATION_AIRPORT_BWI,DESTINATION_AIRPORT_CLE,DESTINATION_AIRPORT_CLT,DESTINATION_AIRPORT_DAL,DESTINATION_AIRPORT_DCA,DESTINATION_AIRPORT_DEN,DESTINATION_AIRPORT_DFW,DESTINATION_AIRPORT_DTW,DESTINATION_AIRPORT_EWR,DESTINATION_AIRPORT_FLL,DESTINATION_AIRPORT_HNL,DESTINATION_AIRPORT_HOU,DESTINATION_AIRPORT_IAD,DESTINATION_AIRPORT_IAH,DESTINATION_AIRPORT_JFK,DESTINATION_AIRPORT_LAS,DESTINATION_AIRPORT_LAX,DESTINATION_AIRPORT_LGA,DESTINATION_AIRPORT_MCI,DESTINATION_AIRPORT_MCO,DESTINATION_AIRPORT_MDW,DESTINATION_AIRPORT_MIA,DESTINATION_AIRPORT_MKE,DESTINATION_AIRPORT_MSP,DESTINATION_AIRPORT_MSY,DESTINATION_AIRPORT_OAK,DESTINATION_AIRPORT_ORD,DESTINATION_AIRPORT_Other,DESTINATION_AIRPORT_PBI,DESTINATION_AIRPORT_PDX,DESTINATION_AIRPORT_PHL,DESTINATION_AIRPORT_PHX,DESTINATION_AIRPORT_RDU,DESTINATION_AIRPORT_RSW,DESTINATION_AIRPORT_SAN,DESTINATION_AIRPORT_SAT,DESTINATION_AIRPORT_SEA,DESTINATION_AIRPORT_SFO,DESTINATION_AIRPORT_SJC,DESTINATION_AIRPORT_SJU,DESTINATION_AIRPORT_SLC,DESTINATION_AIRPORT_SMF,DESTINATION_AIRPORT_SNA,DESTINATION_AIRPORT_STL,DESTINATION_AIRPORT_TPA
0,12.0,1,4,115,1487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,72.0,1,4,125,1440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1,4,145,2762,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-3.0,1,4,307,1129,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,95.0,1,4

In [20]:
# separate into training and testing data
y = df_ml.DEPARTURE_DELAY
X = df_ml.drop(columns=['DEPARTURE_DELAY'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, stratify=None)

In [21]:
# Scale the data with StandardScaler
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 16
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=5, verbose=2)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/5
797579/797579 - 16s - loss: 4479.8961 - acc: 0.0086
Epoch 2/5
797579/797579 - 16s - loss: 4422.3347 - acc: 0.0086
Epoch 3/5
797579/797579 - 16s - loss: 4414.0467 - acc: 0.0086
Epoch 4/5
797579/797579 - 16s - loss: 4408.5055 - acc: 0.0086
Epoch 5/5
797579/797579 - 16s - loss: 4404.3953 - acc: 0.0086
265860/265860 - 2s - loss: 4473.6250 - acc: 0.0085
Loss: 4473.625022920898, Accuracy: 0.008451816625893116
